In [ ]:
pip install nfl_data_py

In [ ]:
pip install duckdb

In [1]:
import nfl_data_py as nfl  # NFL data retrieval and analysis (via https://pypi.org/project/nfl-data-py/)
import pandas as pd  # Data manipulation and analysis
import duckdb # Used to write SQL inside Python script

In [3]:
# Make dataframe output easier to read
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow horizontal scrolling
pd.set_option('display.max_rows', None)  # Show all rows

# Define lists for years and columns that we can use to filter the large play-by-play data set.
years = [2021,2022,2023]
columns = ['season','game_id','drive','series','series_result','fixed_drive_result','desc','weather','roof','surface','temp','wind',
           'kicker_player_id','kickoff_returner_player_id','penalty','return_team','return_yards','penalty_player_id','penalty_type',
           'penalty_yards','end_yard_line','kickoff_inside_twenty','kickoff_in_endzone','kickoff_out_of_bounds','kickoff_downed',
           'kickoff_fair_catch','kick_distance','fumble_lost','drive_start_yard_line','touchdown','defteam','play_type','play_deleted',
           'solo_tackle_1_player_id','posteam_type','penalty_team','game_half','own_kickoff_recovery','game_seconds_remaining',
           'half_seconds_remaining']

# Save NFL play by play data sets to dataframe
df_game_log = nfl.import_pbp_data(years, columns) # Import play-by-play data set
df_players = nfl.import_seasonal_rosters(years) # Import NFL rosters for each year requested
df_teams = nfl.import_team_desc() # Import NFL team information

df_game_log = df_game_log[df_game_log['play_type'] == 'kickoff']

df_teams

2021 done.
2022 done.
2023 done.
Downcasting floats.


,team_abbr,team_name,team_id,team_nick,team_conf,team_division,team_color,team_color2,team_color3,team_color4,team_logo_wikipedia,team_logo_espn,team_wordmark,team_conference_logo,team_league_logo,team_logo_squared
0,ARI,Arizona Cardinals,3800,Cardinals,NFC,NFC West,#97233F,#000000,#ffb612,#a5acaf,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/ari.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
1,ATL,Atlanta Falcons,200,Falcons,NFC,NFC South,#A71930,#000000,#a5acaf,#a30d2d,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/atl.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
2,BAL,Baltimore Ravens,325,Ravens,AFC,AFC North,#241773,#9E7C0C,#9e7c0c,#c60c30,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/bal.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
3,BUF,Buffalo Bills,610,Bills,AFC,AFC East,#00338D,#C60C30,#0c2e82,#d50a0a,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/buf.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
4,CAR,Carolina Panthers,750,Panthers,NFC,NFC South,#0085CA,#000000,#bfc0bf,#0085ca,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500-dark...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
5,CHI,Chicago Bears,810,Bears,NFC,NFC North,#0B162A,#E64100,#0b162a,#E64100,https://upload.wikimedia.org/wikipedia/commons...,https://a.espncdn.com/i/teamlogos/nfl/500/chi.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
6,CIN,Cincinnati Bengals,920,Bengals,AFC,AFC North,#FB4F14,#000000,#000000,#d32f1e,https://upload.wikimedia.org/wikipedia/commons...,https://a.espncdn.com/i/teamlogos/nfl/500/cin.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
7,CLE,Cleveland Browns,1050,Browns,AFC,AFC North,#FF3C00,#311D00,#a5acaf,#d32f1e,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/cle.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
8,DAL,Dallas Cowboys,1200,Cowboys,NFC,NFC East,#002244,#B0B7BC,#acc0c6,#a5acaf,https://upload.wikimedia.org/wikipedia/commons...,https://a.espncdn.com/i/teamlogos/nfl/500/dal.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflverse-pbp/raw/m...
9,DEN,Denver Broncos,1400,Broncos,AFC,AFC West,#002244,#FB4F14,#00234c,#ff5200,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/den.png,https://github.com/nflverse/nflverse-pbp/raw/m...,https://github.com/nflverse/nflverse-pbp/raw/m...,https://raw.githubusercontent.com/nflverse/nfl...,https://github.com/nflverse/nflvers

In [21]:
# Use SQL to create main kickoff data set
kickoffs = duckdb.sql("""
                         select b.position, 
                                a.desc, 
                                a.return_team, 
                                c.team_name as return_team_name,
                                a.defteam as kicking_team,
                                d.team_name as kicking_team_name
                                
                         from df_game_log a inner join
                              df_players b on a.season=b.season and
                                              a.solo_tackle_1_player_id=b.player_id inner join
                              df_teams c on a.return_team=c.team_abbr inner join
                              df_teams d on a.defteam=d.team_abbr
                            
                         where a.play_type='kickoff'
                         """).df()

kickoffs

,position,desc,return_team,return_team_name,kicking_team,kicking_team_name
0,K,2-M.Crosby kicks 65 yards from GB 35 to SF 0. ...,SF,San Francisco 49ers,GB,Green Bay Packers
1,WR,7-J.Bailey kicks 64 yards from NE 35 to CLE 1....,CLE,Cleveland Browns,NE,New England Patriots
2,DB,9-J.Tucker kicks 56 yards from BAL 35 to GB 9....,GB,Green Bay Packers,BAL,Baltimore Ravens
3,RB,2-T.Bass kicks 64 yards from BUF 35 to NE 1. 8...,NE,New England Patriots,BUF,Buffalo Bills
4,WR,6-A.Rosas kicks 65 yards from NO 35 to NYG 0. ...,NYG,New York Giants,NO,New Orleans Saints
5,LB,5-J.Myers kicks 55 yards from SEA 35 to SF 10....,SF,San Francisco 49ers,SEA,Seattle Seahawks
6,TE,2-D.Carlson kicks 59 yards from LV 35 to LAC 6...,LAC,Los Angeles Chargers,LV,Las Vegas Raiders
7,DB,9-R.Gould kicks 59 yards from SF 35 to GB 6. 8...,GB,Green Bay Packers,SF,San Francisco 49ers
8,DB,18-M.Wishnowsky kicks 59 yards from SF 35 to D...,DET,Detroit Lions,SF,San Francisco 49ers
9,DB,14-J.Slye kicks 68 yards from SF 35 to CHI -3....,CHI,Chicago Bears,SF,San Francisco 49ers
